# Marking interesting news


In [1]:
%load_ext autoreload
%autoreload 2
from mongoengine import connect
from tweepyrate import create_apps
from hate_collector.models import Tweet

client = connect("hatespeech-news")
db = client["hatespeech-news"]

We look for tweets that are not replies.

In [2]:
screen_names = [t[1:] for t in db.tweet.distinct('query') if t is not None]
screen_names

['LANACION',
 'abc_es',
 'clarincom',
 'cronica',
 'elmundoes',
 'elpaisuy',
 'infobae',
 'latercera',
 'lavanguardia',
 'perfilcom']

In [5]:
tweets = Tweet.objects(in_reply_to_status_id=None, user__screen_name__in=screen_names)

print(f"There are {tweets.count() / 1e3:.2f}K that are not replies and come from the official news accounts = {['@' + name for name in screen_names]}")

There are 15.18K that are not replies and come from the official news accounts = ['@LANACION', '@abc_es', '@clarincom', '@cronica', '@elmundoes', '@elpaisuy', '@infobae', '@latercera', '@lavanguardia', '@perfilcom']


## News and its replies

In [325]:
tweets_and_replies = db.tweet.aggregate([ 

    {
        "$match": {
            "checked": False,
            "interesting": False,
            "user.screen_name": {"$in": screen_names },
            "in_reply_to_status_id": None
        }
    },

    { 
        "$lookup": {   
            "from": "tweet",   
            "localField": "_id",   
            "foreignField": "in_reply_to_status_id", 
            "as": "replies"
        }
    },
    { 
        "$sample": { 
            "size": 100,
        } 
    }
#    {
#        "$match": {
#            "$expr": {"$gt": [{"$size": "$replies"}, 10]},
#            #"replies": { "$gt": {"$size": 5 } },
#        }
#    }
])


In [326]:
tweets_and_replies = list(tweets_and_replies)
current_idx = 0

In [433]:
from pprint import pprint as ppr

def show_tweet(tweet):
    print(f"{'*' * 80} \n" * 4)
    print(f"Tweet id: {tweet['_id']}")
    print(f"Interesting? {tweet['interesting']}")
    print(f"Checked ? {tweet['checked']}")
    print(tweet["created_at"], "\n\n")
    
    
    print(f"@{tweet['user']['screen_name']}")
    print(tweet["text"])
    
    print("\nRespuestas:")
    
    for j, reply in enumerate(tweet["replies"]):
        print(f"{reply['_id']:<14} -- {j+1:<3}- {reply['text']}")
        
def set_interesting(tweet):                                                                           
    print("Interesting!!!\n")
    tweet["interesting"] = True
    tweet["checked"] = True
    
    print(tweet["text"])                                                                                    
    
    db.tweet.update_one(
        {"_id": tweet["_id"]},
        {"$set": {"interesting": True, "checked": True}}
    )
    
def check(tweet):
    print("Boring\n")
    tweet["interesting"] = False
    tweet["checked"] = True
    
    print(tweet["text"])                                                                                    
    
    db.tweet.update_one(
        {"_id": tweet["_id"]},
        {"$set": {"interesting": False, "checked": True}}
    )
    

In [434]:
print(f"Hay {len(tweets_and_replies)} tweets para chequear")

Hay 100 tweets para chequear


In [459]:
tweet = tweets_and_replies[current_idx]
show_tweet(tweet)

******************************************************************************** 
******************************************************************************** 
******************************************************************************** 
******************************************************************************** 

Tweet id: 1225106309638426627
Interesting? False
Checked ? False
2020-02-05 17:18:00 


@latercera
🔴#LaTerceraPM | La polémica solicitud del Presidente de Ecuador: Reformar ley para “expulsar a extranjeros que vienen a delinquir" https://t.co/GupNF2m4qD https://t.co/TLG2hIq6ub

Respuestas:
1225156484746575875 -- 1  - @latercera Lo mismo deberían haber hecho en Chil con los extranjeros, motochorros, cicarios, prestamistas usureros, ladrones....
1225164151653388289 -- 2  - @latercera Si es eso lo que hay que hacer!!! Cuál es la polémica??? Van a decir que eso es violencia??
1225171068735971328 -- 3  - @latercera Polémica genera que los medios no encuentren absoluta

In [458]:
#set_interesting(tweet)
check(tweet)

current_idx += 1


Boring

El régimen iraní arrestó a 24 personas por difundir rumores sobre la propagación del coronavirus en el país https://t.co/eqrZsL8fOF
